In [5]:
train_data_path = './data/image/train'
test_data_path = './data/image/test'

In [6]:
import os

file_list = []
for root, dirs, files in os.walk(test_data_path):
    for file in files:
        file_list.append(file)

In [7]:
import math
import random
import numpy as np
import pandas as pd
from PIL import Image

import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transform
import torchvision.models as models

In [ ]:
df = pd.read_csv('./data/train.csv')
# print (len(df))
print(df['Image'][0:10])
print(df['Cloth_label'][0:10])
df = df.sample(frac=1)
print(df['Image'][0:10])
print(df['Cloth_label'][0:10])
# print(len(df)) # 20992
# print(df[0:10]['Cloth_label'][1:3])
# print(file_list)

In [ ]:
class trainDataset(Dataset):
    # rootdir = train_data_path =  './data/image/train'
    def __init__(self, root_dir, df, transform):  
        self.root_dir          = root_dir
        self.transform         = transform
        self.df                = df
    
    def __getitem__(self, idx):
        # 不加iloc会报错
        image_label = self.df['Cloth_label'].iloc[idx]
        
        filename = self.df['Image'].iloc[idx]
        image = os.path.join(self.root_dir, filename)
        
        image = Image.open(image).convert('RGB')
        image = self.transform(image)  # preprocessing the image
        
        return image, image_label
    
    def __len__(self):
        # 100000
        return len(self.df)

In [ ]:
class testDataset(Dataset):

    def __init__(self, root_dir, file_list, transform):
        self.root_dir          = root_dir
        self.file_list         = file_list
        self.transform         = transform
    
    def __getitem__(self, idx):
        
        image_label = 0 # 这个没有用，设成0就行了
        
        image = os.path.join(self.root_dir, self.file_list[idx])
        
        image = Image.open(image).convert('RGB')
        image = self.transform(image)  # preprocessing the image
        
        return image, image_label
    
    def __len__(self):
        # 10000
        return len(self.file_list)

In [ ]:
def train(model, train_loader, loss_func, optimizer, device):
    total_loss = 0
    for i, (images, targets) in enumerate(train_loader):
        images = images.to(device)
        targets = targets.to(device)
        
        outputs = model(images)
        loss = loss_func(outputs, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        if (i+1)%1000 == 0:
            print('step [{}/{}] train loss: {}'.format(i+1, len(train_loader), loss.item()))
    return total_loss/len(train_loader)

In [ ]:
def evaluate(model, val_loader, device):
    model.eval()
    with torch.no_grad():
        correct = 0
        total_size = 0
        for i, (images, targets) in enumerate(val_loader):
            images = images.to(device)
            targets = targets.to(device)

            outputs = model(images)
            _, predict = torch.max(outputs, dim=1)

            correct = correct + (predict==targets).sum().item()
            total_size = total_size + targets.size(0)
            
            if (i+1)%100 == 0:
                print('step [{}/{}] val accuracy loss: {}'.format(i+1, len(val_loader), correct/total_size))
                
        accuracy = correct/total_size
        print('Accuracy on val Set: {:.4f} %'.format(100 * accuracy))
        return accuracy

In [ ]:
def fit(model, num_epochs, optimizer, device, lr):
    loss_func = nn.CrossEntropyLoss()
    
    model.to(device)
    loss_func.to(device)
    
    losses = []
    accs = []
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}:'.format(epoch+1, num_epochs))
        loss = train(model, train_loader, loss_func, optimizer, device)
        print('loss on train data: ', loss)
        losses.append(loss)
        
        if (epoch+1)%4 == 0:
            acc = evaluate(model, val_loader, device)
            accs.append(acc)
            print('accuracy on val data: ', acc)
        
        if (epoch + 1) % 10 == 0:
            lr = lr/2
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr


In [ ]:
train_transform = transform.Compose([
    transform.Resize(256),
    transform.RandomHorizontalFlip(),
    transform.RandomVerticalFlip(),
    transform.RandomRotation(30), 
    transform.RandomCrop(224, padding=4),
    transform.ToTensor(),
    transform.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])
])
val_transform = transform.Compose([
    transform.Resize(224),
    transform.ToTensor(),
    transform.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])
])

train_dataset = trainDataset(
    root_dir=train_data_path,
    df=df[0:19000],
    transform=train_transform)
val_dataset = trainDataset(
    root_dir=train_data_path,
    df=df[19000:len(df)],
    transform=val_transform)
test_dataset = testDataset(
    root_dir=test_data_path,
    file_list=file_list,
    transform=val_transform)

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=64,
    shuffle=True
)
val_loader = torch.utils.data.DataLoader(
    dataset=val_dataset,
    batch_size=64,
    shuffle=False
)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=64,
    shuffle=False
)

In [ ]:
device = torch.device('cuda:1')
resnet101 = models.resnet101(pretrained=True)

# for para in list(resnet101.parameters()):
#     para.requires_grad = False
# for para in list(resnet101.layer1.parameters()):
#     para.requires_grad = True
# for para in list(resnet101.layer3.parameters()):
#     para.requires_grad = True
# for para in list(resnet101.layer4.parameters()):
#     para.requires_grad = True
    
num_input = resnet101.fc.in_features
resnet101.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_input, 8)
)
params_to_update = []
for name,param in resnet101.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
#         print(name)

In [ ]:
num_epochs = 20
lr = 0.001

optimizer = torch.optim.SGD(params_to_update, lr=lr, momentum=0.9)
fit(resnet101, num_epochs, optimizer, device, lr)

In [ ]:
data = pd.DataFrame({'Image': file_list})
device = torch.device('cuda:1')
def writeData(model, test_loader, device):
    # write the result
    model.to(device)
    model.eval()
    data_to_save = []
    # context-manager that disabled gradient computation
    with torch.no_grad():
        for i, (images, targets) in enumerate(test_loader):
            # device: cpu or gpu
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, dim=1)
#           print(predicted.cpu().numpy())
            data_to_save.extend(predicted.cpu().numpy())
    data['Cloth_label'] = data_to_save
    data.to_csv('./sample_submission.csv', index=False)
writeData(resnet101, test_loader, device)